In [ ]:
import cv2
from os import getcwd, listdir, path
import sklearn
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

IMAGE_SIZE = 128
BATCH_SIZE = 16
VERBOSE = 1

class_names = ["no", "yes"]
base_path = "../input/brain-tumor-detection"

x_train=[]
y_train=[]

for i in class_names:
    folderPath = os.path.join(base_path,i)
    for j in tqdm(os.listdir(folderPath), ncols=70):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
        x_train.append(img)
        y_train.append(i)
print('Training dataset Loading complete.')

# Loading Testing Datasets from folders
#for i in class_names:
    #folderPath = os.path.join(os.path.join(base_path, 'Testing'), i)
    #for j in tqdm(os.listdir(folderPath), ncols=70):
        #img = cv2.imread(os.path.join(folderPath, j))
        #img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
        #x_train.append(img)
        #y_train.append(i)

aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,horizontal_flip=True, fill_mode="nearest")

x_train = np.array(x_train)
y_train = np.array(y_train)
x_train, y_train = sklearn.utils.shuffle(x_train, y_train, random_state=0)

print('Testing dataset Loading complete.')

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, random_state=47, test_size=0.10)

y_train_new = [class_names.index(i) for i in y_train]
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)
y_test_new = [class_names.index(i) for i in y_test]
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
inputShape = (IMAGE_SIZE, IMAGE_SIZE, 3)
xception = Xception(weights="../input/pretrained-model/xception_weights_tf_dim_ordering_tf_kernels_notop.h5", input_shape=inputShape, include_top=False)
model = xception.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)
model = tf.keras.layers.Dense(2, activation='softmax')(model)
model = tf.keras.models.Model(inputs=xception.input, outputs=model)

tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("xception.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,mode='auto',verbose=VERBOSE)

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
          
validation_data=(x_test, y_test), steps_per_epoch=len(x_train) // BATCH_SIZE,
epochs=3, callbacks=[reduce_lr, checkpoint, tensorboard])
result = model.evaluate(x_test, y_test)
print(result)

In [ ]:
from PIL import Image

In [ ]:
img = cv2.imread('../input/brain-tumor-detection/no/No14.jpg')
img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
img1=np.array(img)
img1=img1.reshape(1,128,128,3)
img1.shape

In [ ]:
plt.imshow(img)
predictions=model.predict(img1)
predict=np.argmax(predictions[0])
if predict==1:
    print("Tumor Detected")
else:
    print("No tumor detected")

In [ ]:
model.save("tumor.h5")

In [ ]:
pip install anvil-uplink

In [ ]:
import anvil.server 

anvil.server.connect('HVRN6VUDKEZYL5R72CNKLJMQ-6ZWZH46YRFHG5ZKT')

In [ ]:
import anvil.media

@anvil.server.callable
def brain_tumor_detect(file):
    with anvil.media.TempFile(file) as filename:
        img = cv2.imread(filename)
    img = cv2.resize(img, (128,128))
    arr=np.array(img)
    img=img.reshape(1,128,128,3)
    predictions=model.predict(img)
    predict=np.argmax(predictions[0])
    return ('Tumor Detected'if predict==1 else 'No Tumor Detected',float(predict))

In [ ]:
anvil.server.wait_forever()